# Исследование надежности заемщиков


### Шаг 1. Изучение общей информации

**Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`**

In [ ]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.  Предобработка данных

#### 2.1 Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца.**

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

#### 2.2 Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце: заменим все отрицательные значения положительными.**

In [ ]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Выведем перечень уникальных значений столбца `children`.**

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма**

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

#### 2.3 Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедились, что все пропуски заполнены. Ещё раз выведем количество пропущенных значений для каждого столбца**

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### 2.4 Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный**

In [ ]:
data['total_income'] = data['total_income'].astype(int)

#### 2.5 Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру**

In [ ]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных**

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

#### 2.6 Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуем данные и ответим на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
data.groupby('children')['debt'].agg(['count', 'sum', 'mean'])

,count,sum,mean
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод:** Незначительное увеличение должников с увеличением количества детей в семье. Но слишком мало данных для анализа семей с тремя  детьми и более.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data.groupby('family_status')['debt'].agg(['count', 'sum', 'mean']).sort_values(by='mean')

,count,sum,mean
family_status,,,
вдовец / вдова,951,63,0.066246
в разводе,1189,84,0.070648
женат / замужем,12261,927,0.075606
гражданский брак,4134,385,0.093130
Не женат / не замужем,2796,273,0.097639


In [ ]:
data.pivot_table(index=['family_status', 'purpose_category'], 
                 values='debt',
                aggfunc=['count', 'sum', 'mean']).sort_values(by=( 'mean', 'debt'))

count  sum      mean
                                                debt debt      debt
family_status         purpose_category                             
вдовец / вдова        операции с недвижимостью   535   28  0.052336
в разводе             операции с недвижимостью   672   46  0.068452
женат / замужем       операции с недвижимостью  6970  486  0.069727
в разводе             получение образования      238   17  0.071429
                      операции с автомобилем     279   21  0.075269
вдовец / вдова        получение образования      198   15  0.075758
гражданский брак      проведение свадьбы        2313  183  0.079118
Не женат / не замужем операции с недвижимостью  1586  129  0.081337
женат / замужем       операции с автомобилем    2717  226  0.083180
                      получение образования     2574  215  0.083528
вдовец / вдова        операции с автомобилем     218   20  0.091743
гражданский брак      операции с недвижимостью   988   91  0.092105
Не женат / не замужем получение образования      575   62  0.107826
гражданский брак      операции с автомобилем     430   51  0.118605
Не женат / не замужем операции с автомобилем     635   82  0.129134
гражданский брак      получение образования      403   60  0.148883

**Вывод:** Из выше приведенных данных видно что меньше всего должников в группе "вдовец / вдова", а больше всего "не женат / не замужем"

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data.groupby('total_income_category')['debt'].agg(['count', 'sum', 'mean']).sort_values(by='mean')

,count,sum,mean
total_income_category,,,
D,349,21,0.060172
B,5014,354,0.070602
A,25,2,0.080000
C,15921,1353,0.084982
E,22,2,0.090909


In [ ]:
data.pivot_table(index=['total_income_category', 'purpose_category'], 
                 values='debt',
                aggfunc=['count', 'sum', 'mean']).sort_values(by=( 'mean', 'debt'))

count  sum      mean
                                                debt debt      debt
total_income_category purpose_category                             
A                     операции с автомобилем       2    0  0.000000
E                     операции с автомобилем       2    0  0.000000
                      получение образования        5    0  0.000000
                      проведение свадьбы           4    0  0.000000
A                     проведение свадьбы           2    0  0.000000
D                     получение образования       75    2  0.026667
                      операции с недвижимостью   168    6  0.035714
B                     проведение свадьбы         544   28  0.051471
A                     операции с недвижимостью    17    1  0.058824
B                     операции с недвижимостью  2558  172  0.067240
C                     операции с недвижимостью  7997  599  0.074903
B                     получение образования      890   69  0.077528
                      операции с автомобилем    1022   85  0.083170
C                     проведение свадьбы        1729  152  0.087912
D                     проведение свадьбы          34    3  0.088235
C                     операции с автомобилем    3181  305  0.095882
                      получение образования     3014  297  0.098540
D                     операции с автомобилем      72   10  0.138889
E                     операции с недвижимостью    11    2  0.181818
A                     получение образования        4    1  0.250000

**Вывод:** Люди с доходами категории A и Е обращаются за кредитами очень редко.(**Предположительно** люди с категорией дохода "А" - в связи с достаточным доходом, не нуждаются в кредитах, а категория "Е" не могут позволить себе кредит в связи с недостаточным доходом для покрытия.) В связи с этим предположением и соответствующей нехваткой данных из этих категорий, их можно не учитывать. Показатели в категориях B, C и D колебляться между собой в районе 1.5 %.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum', 'mean']).sort_values(by=( 'mean', 'debt'))

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,10751,780,0.072551
проведение свадьбы,2313,183,0.079118
получение образования,3988,369,0.092528
операции с автомобилем,4279,400,0.093480


**Вывод:**
Исходя из предоставленных данных и построения таблицы с конверсией можно сделать вывод что люди берущие кредит на проведение свадьбы или на операции с недвижимостью чаще ~ на 2% возвращают кредит без задолженностей чем те кто берут кредит на образование или автомобиль.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Предполагаю что значения в days_employed отсутствуют в связи с тем что человек не трудоустраивался официально вообще и соответственно пропущены значения в тех же строчках по столбцу total_income в связи с отсутствием у него дохода (NaN значения в столбце days_employed полностью соответсвуют NaN значениям в столбце total_income). 

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Медианное значение имеет наиболее точный результат с точки зрения математического прогнозна т.к. меньше подвержено влияюнию аномальных значений в следствии чего имеет наименьшее влияние на результат статистического анализа

### Шаг 4: Общий вывод.

**При поиске закономерностей между должниками и факторами влияющие на это можно строить следующие предположения:**

1) Чем больше детей, тем больше опаздывают с выплачиванием кредита, однако также замечается закономерность того, что чем больше детей - тем меньше в целом оформляют кредит.

2) При дальнейшем исследовании следует обратить внимание сразу на три фактора: это количество людей, оформивших кредит, по какой категории они кредит оформляли, и какая часть из них опаздывала с выплачиванием. Следует отметить, что медианное значение заработка у всех групп, кроме вдовцов и вдов, на уровне погрешности (выводы далее также учитывают этот факт).
По статистическому анализу видно, что больше всего выплату задерживают не женатые люди либо люди в *гражданском браке* по категориям получения образования и операций с автомобилем. Сложно сделать однозначный вывод из-за отсутствия определённых данных, но представляется, что люди, находящиеся в гражданском браке, а также незамужние люди, либо недостаточно ответственны (что является очень грубым социологическим выводом и очень скудно выдерживает критику, хоть и имеет место быть), либо, что вероятнее, недостаточно точно планируют свои финансовые расходы. Для однозначного вывода не хватает информации о том, какого размера кредит они оформляют при одинаковых доходах.
Мы видим практическую состоятельность данной логики, когда смотрим на пласт общества, наименее всего опаздывающего с погашением кредитов: люди в годах (вдовцы, замужние или в разводе) меньше всего берут кредит и при этом наиболее точно планируют свои финансовые расходы (исходя от того, что именно от этого зависит своевременность погашения долга).
Тем не менее, прошу заметить ещё раз, что я склоняюсь к тому ответу, что нам критически не хватает информации о размере средств, на которые оформили кредит.
Также необходимо учитывать возможность того, что у граждан старше уже есть квартира и машина и образование при отсутствии нужды "играть свадьбу".

3) Больше всего оформляют кредит граждане со средним уровнем заработка, но больше никакой полезной информации нам такая сводка данных не дает. Если привести таблицу по категории дохода гражданина и по цели кредита, то тут ситуация начинает вырисовываться. Однозначно не хватает колонки с суммой оформленного кредита, но по цели кредита можно ориентировочно предположить эту сумму. И тут с колоссальным отрывом по задолжности появляются люди с низким доходом но с крупной покупкой.



На мой взгляд не достаточно информации для однозначного вывода. В датафрейме как минимум не хватает информации о конкретных целях оформления кредита (например, для чего именно нужна машина), на какую сумму был взят кредит и на какой срок. Отсутствие обозначенных данных является критическим фактором в вопросе формирования однозначных выводов.